Predicting the popularity of a product.
Least popular can be discontinued

In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
from glob import glob
sns.set()
from imblearn.over_sampling import RandomOverSampler
from sklearn.preprocessing import OneHotEncoder,MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.metrics import confusion_matrix,ConfusionMatrixDisplay,roc_curve,roc_auc_score
from sklearn import set_config
set_config(display = "diagram")  


In [4]:
#set the file directory
os.chdir('C:/Users/vetri/OneDrive/Documents/UArizonaEller/Coursework/Fall 2023/MIS545/Project/SalesData')

In [5]:
#read the dataset to dataframe
amazonSaleRepDF = pd.read_excel('Amazon Sale Report.xlsx',sheet_name='Amazon')

In [15]:
######## Do not run######
#map the sheet names to dataframes
amazonSaleRepDF = {sheet_name: amazonSaleRepXL.parse(sheet_name) 
          for sheet_name in amazonSaleRepXL.sheet_names}

In [6]:
#Display summary statistics
amazonSaleRepDF.describe()

,Date,Qty,Amount,ship-postal-code,Unnamed: 22
count,128975,128975.000000,121180.000000,128942.000000,79925.0
mean,2022-05-12 11:49:27.951928576,0.904431,648.561465,463966.236509,0.0
min,2022-03-31 00:00:00,0.000000,0.000000,110001.000000,0.0
25%,2022-04-20 00:00:00,1.000000,449.000000,382421.000000,0.0
50%,2022-05-10 00:00:00,1.000000,605.000000,500033.000000,0.0
75%,2022-06-04 00:00:00,1.000000,788.000000,600024.000000,0.0
max,2022-06-29 00:00:00,15.000000,5584.000000,989898.000000,0.0
std,NaN,0.313354,281.211687,191476.764941,0.0


In [7]:
#Display the structure
amazonSaleRepDF.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 128975 entries, 0 to 128974
Data columns (total 23 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   Order ID            128975 non-null  object        
 1   Date                128975 non-null  datetime64[ns]
 2   Status              128975 non-null  object        
 3   Fulfilment          128975 non-null  object        
 4   Sales Channel       128975 non-null  object        
 5   ship-service-level  128975 non-null  object        
 6   Style               128975 non-null  object        
 7   SKU                 128975 non-null  object        
 8   Category            128975 non-null  object        
 9   Size                128975 non-null  object        
 10  ASIN                128975 non-null  object        
 11  Courier Status      122103 non-null  object        
 12  Qty                 128975 non-null  int64         
 13  currency            121180 no

In [8]:
#print column names as list
amazonSaleRepDF.columns.tolist()

['Order ID',
 'Date',
 'Status',
 'Fulfilment',
 'Sales Channel ',
 'ship-service-level',
 'Style',
 'SKU',
 'Category',
 'Size',
 'ASIN',
 'Courier Status',
 'Qty',
 'currency',
 'Amount',
 'ship-city',
 'ship-state',
 'ship-postal-code',
 'ship-country',
 'promotion-ids',
 'B2B',
 'fulfilled-by',
 'Unnamed: 22']

orderid,date, status(remove 'cancelled orders'), style,qty,

DATA CLEANING (Exploratory Data Analysis)

In [9]:
#remove spaces and special characters from column names
amazonSaleRepDF.columns = amazonSaleRepDF.columns.str.replace('_', '').str.replace('-','').str.replace(' ','')


In [10]:
#change the column names to lower case
amazonSaleRepDF.columns = map(str.lower, amazonSaleRepDF.columns)
print(amazonSaleRepDF.columns.tolist())

['orderid', 'date', 'status', 'fulfilment', 'saleschannel', 'shipservicelevel', 'style', 'sku', 'category', 'size', 'asin', 'courierstatus', 'qty', 'currency', 'amount', 'shipcity', 'shipstate', 'shippostalcode', 'shipcountry', 'promotionids', 'b2b', 'fulfilledby', 'unnamed:22']


In [11]:
#removing unwanted columns
# amazonSaleRepDF.drop(["date","fulfilledby","currency","unnamed:22","promotionids","courierstatus",
#              "shipcountry"],axis ="columns", inplace = True)

#keep the selected columns and drop the rest in the data frame
amazonSaleRepDF1 = amazonSaleRepDF[['orderid','status','style','date','qty','sku']]

In [12]:
#head of the dataframe
amazonSaleRepDF1.head()

,orderid,status,style,date,qty,sku
0,405-8078784-5731545,Cancelled,SET389,2022-04-30,0,SET389-KR-NP-S
1,171-9198151-1101146,Shipped - Delivered to Buyer,JNE3781,2022-04-30,1,JNE3781-KR-XXXL
2,404-0687676-7273146,Shipped,JNE3371,2022-04-30,1,JNE3371-KR-XL
3,403-9615377-8133951,Cancelled,J0341,2022-04-30,0,J0341-DR-L
4,407-1069790-7240320,Shipped,JNE3671,2022-04-30,1,JNE3671-TU-XXXL


In [13]:
#remove rows with status as cancelled
amazonSaleRepDF1 = amazonSaleRepDF1[amazonSaleRepDF1.status != 'Cancelled']

In [14]:
#print 5 rows of the dataframe
amazonSaleRepDF1.head()

,orderid,status,style,date,qty,sku
1,171-9198151-1101146,Shipped - Delivered to Buyer,JNE3781,2022-04-30,1,JNE3781-KR-XXXL
2,404-0687676-7273146,Shipped,JNE3371,2022-04-30,1,JNE3371-KR-XL
4,407-1069790-7240320,Shipped,JNE3671,2022-04-30,1,JNE3671-TU-XXXL
5,404-1490984-4578765,Shipped,SET264,2022-04-30,1,SET264-KR-NP-XL
6,408-5748499-6859555,Shipped,J0095,2022-04-30,1,J0095-SET-L


In [15]:
#check for null values
amazonSaleRepDF1.isnull().sum()

orderid    0
status     0
style      0
date       0
qty        0
sku        0
dtype: int64

In [50]:
######## Do not run######
#replace null values with unknown in the column shipcity and shipstate
# amazonSaleRepDF['shipcity'].fillna('Unknown', inplace=True)
# amazonSaleRepDF['shipstate'].fillna('Unknown', inplace=True)

In [18]:
# drop duplicates if any
amazonSaleRepDF1.drop_duplicates(inplace=True)

In [17]:
amazonSaleRepDF1.head()

,orderid,status,style,date,qty,sku
1,171-9198151-1101146,Shipped - Delivered to Buyer,JNE3781,2022-04-30,1,JNE3781-KR-XXXL
2,404-0687676-7273146,Shipped,JNE3371,2022-04-30,1,JNE3371-KR-XL
4,407-1069790-7240320,Shipped,JNE3671,2022-04-30,1,JNE3671-TU-XXXL
5,404-1490984-4578765,Shipped,SET264,2022-04-30,1,SET264-KR-NP-XL
6,408-5748499-6859555,Shipped,J0095,2022-04-30,1,J0095-SET-L


In [19]:
amazonSaleRepDF1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 110640 entries, 1 to 128974
Data columns (total 6 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   orderid  110640 non-null  object        
 1   status   110640 non-null  object        
 2   style    110640 non-null  object        
 3   date     110640 non-null  datetime64[ns]
 4   qty      110640 non-null  int64         
 5   sku      110640 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(4)
memory usage: 5.9+ MB


In [20]:
amazonSaleRepDF1.columns.tolist()

['orderid', 'status', 'style', 'date', 'qty', 'sku']

In [21]:
#dataframe of repeated orderid & sku and store it in repeatedOrderDF
repeatedOrderDF = amazonSaleRepDF[amazonSaleRepDF.duplicated(['orderid','sku'],keep=False)]

In [95]:
############# verifying the duplicate data #################
#fetch the row with orderid 407-8364731-6449117	
amazonSaleRepDF[amazonSaleRepDF['qty']=='407-8364731-6449117']

,orderid,date,status,fulfilment,saleschannel,shipservicelevel,style,sku,category,size,...,currency,amount,shipcity,shipstate,shippostalcode,shipcountry,promotionids,b2b,fulfilledby,unnamed:22
52330,407-8364731-6449117,2022-05-29,Shipped,Amazon,Amazon.in,Expedited,JNE3769,JNE3769-KR-L,kurta,L,...,NaN,NaN,Theni,TAMIL NADU,625579.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,0.0
52331,407-8364731-6449117,2022-05-29,Shipped,Amazon,Amazon.in,Expedited,JNE3769,JNE3769-KR-XXL,kurta,XXL,...,INR,487.0,Theni,TAMIL NADU,625579.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,0.0
52332,407-8364731-6449117,2022-05-29,Shipped,Amazon,Amazon.in,Expedited,JNE3769,JNE3769-KR-XXXL,kurta,3XL,...,INR,487.0,Theni,TAMIL NADU,625579.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,0.0
52333,407-8364731-6449117,2022-05-29,Shipped,Amazon,Amazon.in,Expedited,JNE3769,JNE3769-KR-L,kurta,L,...,INR,487.0,Theni,TAMIL NADU,625579.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,0.0
52334,407-8364731-6449117,2022-05-29,Shipped,Amazon,Amazon.in,Expedited,JNE3769,JNE3769-KR-XL,kurta,XL,...,INR,487.0,Theni,TAMIL NADU,625579.0,IN,IN Core Free Shipping 2015/04/08 23-48-5-108,False,NaN,0.0


In [22]:
#remove rows with status as cancelled
amazonSaleRepPopularDF = amazonSaleRepDF1[amazonSaleRepDF1.status != 'Cancelled']

In [23]:
amazonSaleRepPopularDF.head()

,orderid,status,style,date,qty,sku
1,171-9198151-1101146,Shipped - Delivered to Buyer,JNE3781,2022-04-30,1,JNE3781-KR-XXXL
2,404-0687676-7273146,Shipped,JNE3371,2022-04-30,1,JNE3371-KR-XL
4,407-1069790-7240320,Shipped,JNE3671,2022-04-30,1,JNE3671-TU-XXXL
5,404-1490984-4578765,Shipped,SET264,2022-04-30,1,SET264-KR-NP-XL
6,408-5748499-6859555,Shipped,J0095,2022-04-30,1,J0095-SET-L


In [24]:
#summary statistics of amazonSaleRepPopularDF
amazonSaleRepPopularDF.describe()

,date,qty
count,110640,110640.000000
mean,2022-05-12 14:01:42.299349504,1.003154
min,2022-03-31 00:00:00,0.000000
25%,2022-04-20 00:00:00,1.000000
50%,2022-05-10 00:00:00,1.000000
75%,2022-06-04 00:00:00,1.000000
max,2022-06-29 00:00:00,15.000000
std,NaN,0.100339


In [25]:
# Aggregate the data to find the number of transactions per product (SKU)
product_transactions = amazonSaleRepPopularDF.groupby('sku')['orderid'].count().sort_values(ascending=False)

In [26]:
#define a threshold for popular products
popularThreshold = product_transactions.median()
popularThreshold

7.0

In [27]:
# Create a binary target variable 'Is_Popular'
amazonSaleRepPopularDF['Is_Popular'] = amazonSaleRepPopularDF['sku'].apply(lambda x: 1 if product_transactions[x] > popularThreshold else 0)

In [28]:
# Encode the 'Style' and 'SKU' columns
label_encoder = LabelEncoder()
amazonSaleRepPopularDF['Style_Code'] = label_encoder.fit_transform(amazonSaleRepPopularDF['style'])
amazonSaleRepPopularDF['SKU_Code'] = label_encoder.fit_transform(amazonSaleRepPopularDF['sku'])

In [29]:
amazonSaleRepPopularDF.head()

,orderid,status,style,date,qty,sku,Is_Popular,Style_Code,SKU_Code
1,171-9198151-1101146,Shipped - Delivered to Buyer,JNE3781,2022-04-30,1,JNE3781-KR-XXXL,1,845,4387
2,404-0687676-7273146,Shipped,JNE3371,2022-04-30,1,JNE3371-KR-XL,1,531,2596
4,407-1069790-7240320,Shipped,JNE3671,2022-04-30,1,JNE3671-TU-XXXL,1,752,3790
5,404-1490984-4578765,Shipped,SET264,2022-04-30,1,SET264-KR-NP-XL,1,1230,6207
6,408-5748499-6859555,Shipped,J0095,2022-04-30,1,J0095-SET-L,1,179,650


In [77]:
########## Do not run##########

#encode categorical variables
# label_encoders = {}
# for column in ['status', 'style']:
#     label_encoders[column] = LabelEncoder()
#     amazonSaleRepPopularDF[column] = label_encoders[column].fit_transform(amazonSaleRepPopularDF[column])

In [30]:
#set feature and target variables
features = amazonSaleRepPopularDF[[ 'Style_Code', 'SKU_Code', 'qty']]
target = amazonSaleRepPopularDF['Is_Popular']

In [31]:
# One-hot encoding for categorical features
encoder = OneHotEncoder()
encoded_features = encoder.fit_transform(features)

In [32]:
#split the data into train and test sets with 70:30 ratio
X_train, X_test, y_train, y_test = train_test_split(encoded_features, target, test_size=0.3, random_state=42)

In [33]:
#create and train the random forest classifier model with criterion as entropy
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [96]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import classification_report, accuracy_score

In [97]:
#predict and evaluate the model (entropy)
y_pred = rf_classifier.predict(X_test)
print(confusion_matrix(y_test,y_pred))


[[33091     0]
 [    2    99]]


In [100]:
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

Accuracy: 0.999939744516751
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     33091
           1       1.00      0.98      0.99       101

    accuracy                           1.00     33192
   macro avg       1.00      0.99      0.99     33192
weighted avg       1.00      1.00      1.00     33192



In [103]:
# display the confusion matrix graphically in light colours
cm = confusion_matrix(y_test,y_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm,display_labels=rf_classifier.classes_)
disp.plot(cmap='Pastel1')


X_train shape: (77448, 111801)
X_test shape: (33192, 111801)
